reference for training custom NER 

https://github.com/amrrs/custom-ner-with-spacy3/blob/main/Custom_NER_with_Spacy3.ipynb

In [1]:
!pip install -U spacy -q

In [2]:
!python -m spacy info


============================== Info about spaCy ==============================

spaCy version    3.8.3                         
Location         f:\sfu\python\lib\site-packages\spacy
Platform         Windows-10-10.0.19045-SP0     
Python version   3.9.5                         
Pipelines        en_core_web_sm (3.8.0)        



In [3]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object

In [4]:
import json
with open('healthcare_annotations.json', encoding='utf-8') as f:
    TRAIN_DATA = json.load(f)

In [5]:
#remove null values
filtered_annotations = [item for item in TRAIN_DATA['annotations'] if item is not None]

for text, annot in tqdm(filtered_annotations): 
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents 
    db.add(doc)

db.to_disk("./training_data.spacy") # save the docbin object

100%|██████████| 24/24 [00:00<00:00, 812.88it/s]

Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity


In [11]:
! python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency --force

ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [12]:
! python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     48.62    0.00    0.00    0.00    0.00
 10     200       2593.14   2132.81   66.67   70.27   63.41    0.67
 21     400        272.67    412.89   98.78   98.78   98.78    0.99
 34     600          8.58     12.05  100.00  100.00  100.00    1.00
 53     800         10.53      5.78  100.00  100.00  100.00    1.00
 76    1000          0.00      0.00  100.00  100.00  100.00    1.00
106    1200          0.00      0.00  100.00  100.00  100.00    1.00
143    1400          0.00      0.00  100.00  100.00  100.00    1.00
189    1600        159.83    100.46 

In [16]:
nlp_ner = spacy.load("model-best") 

In [23]:
import pandas as pd
from random import randint

df = pd.read_csv('Datasets/Health_News/Health_News.csv')
df = df.dropna(subset=['content'])

x = randint(0, 250)

sample_text = df["content"].iloc[x]

doc = nlp_ner(sample_text)

In [24]:
spacy.displacy.render(doc, style="ent", jupyter=True)